In [ ]:
import os
import glob
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BertTokenizer, BertForSequenceClassification
from tqdm import tqdm

# ==========================================
# [설정 구역] 경로 확인 필수!
# ==========================================
# 1. 데이터가 있는 폴더 (csv 파일들이 있는 곳)
INPUT_FOLDER = r"/content/drive/MyDrive/news_clean/BGF리테일_clean.csv"

# 2. 결과 저장할 폴더
OUTPUT_FOLDER = r"C:\Users\PC\Desktop\project\esg_results"

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"▶ 현재 작업 장치: {device} (GPU가 없으면 속도가 느릴 수 있습니다)")

# ==========================================
# [모델 로딩] 1. 번역 모델 & 2. ESG 모델
# ==========================================
print("▶ 모델을 로드 중입니다... (처음 실행 시 몇 분 정도 걸립니다)")

# 1. 한글 -> 영어 번역 모델 (Helsinki-NLP)
trans_model_name = "Helsinki-NLP/opus-mt-ko-en"
trans_tokenizer = AutoTokenizer.from_pretrained(trans_model_name)
trans_model = AutoModelForSeq2SeqLM.from_pretrained(trans_model_name).to(device)

# 2. ESG 분류 모델 (FinBERT-ESG)
esg_model_name = "yiyanghkust/finbert-esg"
esg_tokenizer = BertTokenizer.from_pretrained(esg_model_name)
esg_model = BertForSequenceClassification.from_pretrained(esg_model_name).to(device)

# ESG 라벨 정의
labels_map = {0: 'None', 1: 'Environmental', 2: 'Social', 3: 'Governance'}

# ==========================================
# [함수] 번역 및 분류 로직
# ==========================================
def translate_ko_to_en(text_list):
    """
    한글 텍스트 리스트를 받아 영어로 번역하여 반환
    """
    # 텍스트가 너무 길면 자름 (번역 모델 제한)
    inputs = trans_tokenizer(text_list, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        translated_tokens = trans_model.generate(**inputs, max_length=512)
    return trans_tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

def classify_esg_batch(english_texts):
    """
    영어 텍스트 리스트를 받아 ESG 라벨과 확률 반환
    """
    inputs = esg_tokenizer(english_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = esg_model(**inputs)

    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    results = []
    for prob in probs:
        max_idx = torch.argmax(prob).item()
        results.append((labels_map[max_idx], prob[max_idx].item()))
    return results

# ==========================================
# [실행] 전체 파일 처리 루프
# ==========================================
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

# CSV 파일 찾기
csv_files = glob.glob(os.path.join(INPUT_FOLDER, "*.csv"))
print(f"▶ 총 {len(csv_files)}개의 파일을 찾았습니다.")

if not csv_files:
    print("!! 파일을 찾지 못했습니다. 경로를 다시 확인해주세요 !!")
    print(f"입력 경로: {INPUT_FOLDER}")
else:
    # 파일 하나씩 처리
    for file_path in tqdm(csv_files, desc="전체 파일 진행률"):
        try:
            filename = os.path.basename(file_path)

            # 1. 파일 읽기 (인코딩 자동 감지 시도)
            try:
                df = pd.read_csv(file_path, encoding='utf-8')
            except UnicodeDecodeError:
                df = pd.read_csv(file_path, encoding='cp949')

            # 2. 본문 컬럼 찾기
            target_col = None
            for col in ['본문', 'content', 'text', '기사내용']:
                if col in df.columns:
                    target_col = col
                    break

            if target_col is None:
                # 본문 컬럼이 없으면 건너뜀
                continue

            # 데이터가 비어있으면 건너뜀
            df = df.dropna(subset=[target_col])
            if len(df) == 0:
                continue

            # 3. 배치 처리 (속도를 위해 8개~16개씩 묶어서 처리)
            batch_size = 8  # 메모리 부족 시 이 숫자를 줄이세요 (예: 4)
            texts = df[target_col].astype(str).tolist()

            esg_labels = []
            esg_confidences = []

            for i in range(0, len(texts), batch_size):
                batch_texts = texts[i : i + batch_size]

                # [단계 1] 번역 (한글 -> 영어)
                translated_batch = translate_ko_to_en(batch_texts)

                # [단계 2] 분류 (영어 -> ESG)
                classified_batch = classify_esg_batch(translated_batch)

                for label, conf in classified_batch:
                    esg_labels.append(label)
                    esg_confidences.append(conf)

            # 4. 결과 저장
            df['ESG_Label'] = esg_labels
            df['Confidence'] = esg_confidences

            save_path = os.path.join(OUTPUT_FOLDER, "esg_" + filename)
            df.to_csv(save_path, index=False, encoding='utf-8-sig')

        except Exception as e:
            print(f"\n[오류 발생] 파일명: {filename} / 내용: {e}")
            continue

    print(f"\n▶ 모든 작업이 완료되었습니다! 결과 폴더: {OUTPUT_FOLDER}")

▶ 현재 작업 장치: cpu (GPU가 없으면 속도가 느릴 수 있습니다)
▶ 모델을 로드 중입니다... (처음 실행 시 몇 분 정도 걸립니다)
▶ 총 0개의 파일을 찾았습니다.
!! 파일을 찾지 못했습니다. 경로를 다시 확인해주세요 !!
입력 경로: /content/drive/MyDrive/news_clean/BGF리테일_clean


In [ ]:
# 종목 하나 돌리기

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BertTokenizer, BertForSequenceClassification
from tqdm import tqdm
import os

# =================================================================================
# [1] 설정: 분석할 파일 경로를 여기에 정확히 입력하세요.
# =================================================================================
# 경로 앞에 r을 붙여야 오류가 안 납니다.
FILE_PATH = r"/content/drive/MyDrive/news_clean/BGF리테일_clean.csv"

# 결과를 저장할 경로 (바탕화면 등으로 설정해도 됩니다)
SAVE_PATH = r"/content/drive/MyDrive/news_clean/BGF리테일_esg_result.csv"

# 빠른 테스트를 위해 상위 5개만 먼저 해볼까요? (전체 다 하려면 False로 변경)
TEST_MODE = False

# =================================================================================
# [2] 모델 준비 (자동으로 다운로드 됩니다)
# =================================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"▶ 사용 장치: {device}")

print("1. 번역 모델 로딩 중... (Helsinki-NLP/opus-mt-ko-en)")
trans_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ko-en")
trans_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ko-en").to(device)

print("2. ESG 모델 로딩 중... (yiyanghkust/finbert-esg)")
esg_tokenizer = BertTokenizer.from_pretrained("yiyanghkust/finbert-esg")
esg_model = BertForSequenceClassification.from_pretrained("yiyanghkust/finbert-esg").to(device)

labels_map = {0: 'None', 1: 'Environmental', 2: 'Social', 3: 'Governance'}

# =================================================================================
# [3] 처리 함수 정의
# =================================================================================
def analyze_one_text(text):
    if not isinstance(text, str) or len(text.strip()) < 2:
        return "None", 0.0

    # 1. 번역 (한글 -> 영어)
    # 문장이 너무 길면 앞부분 512 토큰만 사용
    inputs = trans_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        translated = trans_model.generate(**inputs, max_length=512)
    eng_text = trans_tokenizer.decode(translated[0], skip_special_tokens=True)

    # 2. 분류 (영어 -> ESG)
    esg_inputs = esg_tokenizer(eng_text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = esg_model(**esg_inputs)

    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    max_idx = torch.argmax(probs).item()
    confidence = probs[0][max_idx].item()

    return labels_map[max_idx], confidence

# =================================================================================
# [4] 실행
# =================================================================================
print(f"\n▶ 파일 읽기 시작: {FILE_PATH}")

try:
    # 인코딩 문제 해결을 위해 utf-8 시도 후 실패 시 cp949 시도
    try:
        df = pd.read_csv(FILE_PATH, encoding='utf-8')
    except UnicodeDecodeError:
        print("utf-8 읽기 실패, cp949로 재시도합니다.")
        df = pd.read_csv(FILE_PATH, encoding='cp949')

    print(f"데이터 로드 성공! 총 {len(df)}개의 행이 있습니다.")

    # 테스트 모드면 5개만 자르기
    if TEST_MODE:
        print("※ 테스트 모드입니다. 상위 5개만 처리합니다.")
        df = df.head(5)

    # 분석할 컬럼 찾기 (본문, content, text 중 하나)
    target_col = None
    for col in ['본문', 'content', 'text', '기사내용']:
        if col in df.columns:
            target_col = col
            break

    if target_col is None:
        raise ValueError("파일에 '본문' 또는 'content' 컬럼이 없습니다. 컬럼명을 확인해주세요.")

    print(f"분석할 컬럼: {target_col}")

    # 반복문으로 처리
    esg_labels = []
    confidences = []

    for i, row in tqdm(df.iterrows(), total=len(df), desc="분석 진행 중"):
        text = row[target_col]
        label, score = analyze_one_text(text)
        esg_labels.append(label)
        confidences.append(score)

    # 결과 저장
    df['ESG_Label'] = esg_labels
    df['Confidence'] = confidences

    df.to_csv(SAVE_PATH, index=False, encoding='utf-8-sig')
    print(f"\n▶ 성공! 결과가 저장되었습니다: {SAVE_PATH}")

except Exception as e:
    print(f"\n[오류 발생] {e}")

▶ 사용 장치: cuda
1. 번역 모델 로딩 중... (Helsinki-NLP/opus-mt-ko-en)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/842k [00:00<?, ?B/s]

...


▶ 파일 읽기 시작: /content/drive/MyDrive/news_clean/BGF리테일_clean.csv
데이터 로드 성공! 총 2483개의 행이 있습니다.
분석할 컬럼: 본문


분석 진행 중:   0%|          | 0/2483 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

분석 진행 중: 100%|██████████| 2483/2483 [23:08<00:00,  1.79it/s]


▶ 성공! 결과가 저장되었습니다: /content/drive/MyDrive/news_clean/BGF리테일_esg_result.csv


In [ ]:

import torch
torch.cuda.is_available()

True

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/news_clean/BGF리테일_esg_result.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2483 entries, 0 to 2482
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   일자          2483 non-null   int64  
 1   제목          2483 non-null   object 
 2   본문          2483 non-null   object 
 3   ticker      2483 non-null   int64  
 4   ESG_Label   1512 non-null   object 
 5   Confidence  2483 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 116.5+ KB


In [ ]:
df['ESG_Label'].value_counts()

,count
ESG_Label,
Social,1300
Environmental,187
Governance,25


In [ ]:
gov = df.loc[df['ESG_Label'] == 'Governance']
gov

,일자,제목,본문,ticker,ESG_Label,Confidence
102,20150723,"BGF리테일, 협력사 지원 1200억 규모 '상생결제시스템' 도입",[BGF리테일이 협력사와의 동반성장을 위해 4개 시중은행과 ‘상생결제시스템’ 도입을...,282330,Governance,0.520855
215,20151215,BGF리테일 홍석조 회장 장남 전무로 승진,[데일리안 = 김영진 기자]BGF리테일은 15일 2016년 정기인사를 단행하며 홍석...,282330,Governance,0.757438
224,20160104,"BGF리테일, 신규 임원 특별채용 외부전문가 영입",장영철(왼쪽) BGF리테일 인사총무실장과 오정후 전략기획실장 (사진=BGF리테일 제...,282330,Governance,0.490045
226,20160104,BGF리테일 신규 임원 특별 채용,편의점 CU를 운영하고 있는 BGF리테일은 인사총무실장과 전략기획실장 직책에 각각 ...,282330,Governance,0.637230
460,20161027,"BGF리테일, 602개 기관에 IR 시장에 적극적인 피드백",{IL01} 편의점 'CU'를 운영하는 BGF리테일은 업계 최초로 IR 전담팀을 만...,282330,Governance,0.462863
497,20161212,"BGF리테일, 2017년 정기 인사 조용준 류철한 상무 승진","왼쪽부터 BGF 4권역장 조용준 상무, 재무지원실장 류철한 상무, BGF 계열사인 ...",282330,Governance,0.434143
623,20170609,"BGF리테일, 지주사 체제 전환으로 대주주 지배력↑-삼성",분할 전 BGF리테일 주주 구성.[이데일리 이명철 기자] 삼성증권은 9일 BGF리테...,282330,Governance,0.839425
742,20171024,"BGF리테일 '2세 경영' 본격화...홍정국 전무, 부사장 승진",편의점 CU를 운영하는 BGF리테일이 지주사 체제 전환과 함께 홍정국 ‘2세 경영’...,282330,Governance,0.739566
746,20171024,"`지주사 전환` 앞둔 BGF리테일, 임원인사 단행 이건준 대표 사장 승진",지주사 전환을 앞둔 BGF리테일이 투자회사인 BGF의 신임 대표이사(사장)로 이건준...,282330,Governance,0.664383
749,20171024,"'지주사 전환' BGF리테일, '2017년 임원인사' 단행",[스포츠서울 김자영기자] BGF리테일이 24일 ‘2017년 임원인사’를 단행했다. ...,282330,Governance,0.487600


In [ ]:
gov.to_csv('/content/drive/MyDrive/가격데이터_단기/BGF리테일_gov_result.csv', index=False, encoding='utf-8-sig')

In [ ]:
env = df.loc[df['ESG_Label'] == 'Environmental']
env

,일자,제목,본문,ticker,ESG_Label,Confidence
1,20150112,[동정] 홍석조 BGF리테일 회장 비전 선포식 外,홍석조 BGF리테일 회장 비전 선포식홍석조(62 사진) BGF리테일 회장은 지난 9...,282330,Environmental,0.968495
19,20150210,"BGF리테일, 협력사 정산대금 조기 지급",\n \n [뉴스핌=강필성 기자] BGF리테일은 설을 앞두고 상품 및 물류 중소...,282330,Environmental,0.878950
86,20150611,"상장 1년 만에 주가 3배로 뛴 BGF리테일, 매장 PB상품 확대 ""성장 수익 두 ...",[ 윤정현 기자 ] 유가증권시장에 상장한 지 갓 1년을 넘긴 BGF리테일 주가가 \...,282330,Environmental,0.461566
164,20151016,"BGF리테일, CU 편의점주 위한 ‘힐링여행’ 진행","[뉴스핌=강필성 기자] BGF리테일의 편의점 CU는 협력사와 함께 지난 여름, 음료...",282330,Environmental,0.751529
236,20160112,"BGF리테일, PB 통합 브랜드 '헤이루' 선보여",BGF리테일은 자체브랜드(PB)상품의 통합 브랜드 '헤이루(HEYROO)'를 론칭한...,282330,Environmental,0.935503
...,...,...,...,...,...,...
2418,20240903,"BGF리테일, 업계 최대 규모 부산 물류센터 첫 삽...2026년 가동",BGF리테일 물류센터 조감도. 부산시 제공 \n[파이낸셜뉴스] 편의점 CU를 운영하...,282330,Environmental,0.954770
2433,20241007,"해외로 뻗는 CU BGF리테일, 몽골 카자흐서 'K-편의점' 신화 쓰는 중",아시아투데이 서병주 기자 = 국내 편의점 '빅2' 중 하나인 CU를 운영하는 BGF...,282330,Environmental,0.963356
2456,20241107,"BGF리테일, 3분기 영업익 전년比 4.8% 증가 “사업 효율화 추진 결과”",BGF리테일은 지난 3분기 연결 기준 영업이익이 전년 동기 대비 4.8% 증가한 9...,282330,Environmental,0.475278
2465,20241107,BGF리테일 3분기 영업익 912억원 전년 대비 4.8% 증가,편의점 CU를 운영하는 BGF리테일은 연결기준 올해 3분기 영업이익이 912억원으로...,282330,Environmental,0.523057
